<div class="alert alert-danger">
**Due date:** 2018-02-16
</div>

# L4: Syntactic analysis

## Introduction

Syntactic analysis, also called syntactic parsing, is the task of mapping a sentence to a formal representation of its syntactic structure. In this lab you will implement a syntactic parser that produces **dependency trees**. A dependency tree consists of directed arcs between individual words (tokens) of a sentence. To see some examples, have a look at the [Syntax: General Principles](http://universaldependencies.org/u/overview/syntax.html) page of the [Universal Dependencies Project](http://universaldependencies.org/).

Your starting point for this lab is a complete reference implementation of a syntactic parser in the Python module `nlp4`. This system (which consists of roughly 200 lines of code) implements a simple pipeline architecture that includes a variant of the part-of-speech tagger that you implemented in lab&nbsp;3 and a transition-based dependency parser based on the arc-standard algorithm.

In [3]:
import nlp4

The reference system has the following components:

1. code to read in dependency trees in the [CoNLL-U Format](http://universaldependencies.org/format.html)
2. a class that implements a multi-class perceptron classifier (lab&nbsp;1)
3. a class that implements a part-of-speech tagger (lab&nbsp;3)
4. a class that implements a transition-based dependency parser (this lab)
5. code to train and evaluate the parser on a treebank in the CoNLL-U format

These components are explained in more detail below.

<div class="alert alert-warning">
For this lab you only need to implement the functions that read dependency trees (Problem&nbsp;1) and the class that implements the parser (Problem&nbsp;2). As for the other components, you will import them from the reference system.
</div>

## Read data

The data set that you will be using in this lab is the [English Web Treebank](http://universaldependencies.org/en/overview/introduction.html) from the [Universal Dependencies Project](http://universaldependencies.org/). This is a corpus of written English taken from five genres of web media: weblogs, newsgroups, emails, reviews, and Yahoo! answers. Each sentence in the corpus was semi-automatically annotated with (among other things) part-of-speech tags and syntactic dependency trees.

In the original files from the Universal Dependencies Project, annotated sentences are stored in the [CoNLL-U Format](http://universaldependencies.org/format.html). This is a simple text-based format where a sentence is represented with one line per word (token), and where each line contains 10 tab-separated fields with various pieces of information about the word in question &ndash; including the word form and the part-of-speech tag.

The following code cell prints a concrete example, which shows how the data looks like when it comes to Python:

In [4]:
with open("/home/TDDE09/labs/l4/data/en-ud-dev.conllu") as fp:
    print(next(nlp4.conllu(fp)))

[['1', 'From', 'from', 'ADP', 'IN', '_', '3', 'case', '_', '_'], ['2', 'the', 'the', 'DET', 'DT', 'Definite=Def|PronType=Art', '3', 'det', '_', '_'], ['3', 'AP', 'AP', 'PROPN', 'NNP', 'Number=Sing', '4', 'nmod', '_', '_'], ['4', 'comes', 'come', 'VERB', 'VBZ', 'Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin', '0', 'root', '_', '_'], ['5', 'this', 'this', 'DET', 'DT', 'Number=Sing|PronType=Dem', '6', 'det', '_', '_'], ['6', 'story', 'story', 'NOUN', 'NN', 'Number=Sing', '4', 'nsubj', '_', '_'], ['7', ':', ':', 'PUNCT', ':', '_', '4', 'punct', '_', '_']]


As you can see, each sentence is represented as a list of lists of strings. The inner lists correspond to the word lines, and the elements of the inner lists correspond to the tab-separated fields. Take a moment to think about how these elements map to the fields as they are described on the [CoNLL-U Format](http://universaldependencies.org/format.html) page. In particular, try to identify the list indices that correspond to the fields holding the word form, part-of-speech tag, and syntactic head of the relevant word.

<div class="panel panel-primary">
<div class="panel-heading">Problem 1</div>
<div class="panel-body">
Implement a generator function `trees()` that reads dependency trees stored in the CoNLL-U format, and for each tree yields a triple consisting of the list of words in the sentence (strings), the corresponding list of part-of-speech tags (strings), and the syntactic heads for all of the words (integers).
</div>
</div>

This is how the output of the function `trees()` should look like for the sample sentence:

In [5]:
with open("/home/TDDE09/labs/l4/data/en-ud-dev.conllu") as fp:
    print(next(nlp4.trees(fp)))

(['<ROOT>', 'From', 'the', 'AP', 'comes', 'this', 'story', ':'], ['<ROOT>', 'ADP', 'DET', 'PROPN', 'VERB', 'DET', 'NOUN', 'PUNCT'], [0, 3, 3, 4, 0, 6, 4, 4])


Note that you are supposed to explicitly convert the head fields from strings to integers. For technical reasons, you are also supposed to pre-pad each sentence with a special pseudo-word `<ROOT>`. The part-of-speech tag of this pseudo-word should be `<ROOT>`, and its head should have index&nbsp;0.

To solve Problem&nbsp;1, you should modify the following code cell. Please note that you are supposed to write this function as a [generator function](https://wiki.python.org/moin/Generators). In particular, the function should *not* return a list of triples. You may use the function `nlp4.conllu()` for reading the raw data.

In [6]:
def trees(source):
    """Reads trees from an input source.
    
    Args:
        source: An iterable, such as a file pointer.
    
    Yields:
        Triples of the form `words`, `tags`, heads where: `words` is
        the list of words of the tree (including the pseudo-word
        <ROOT> at position 0), `tags` is the list of corresponding
        part-of-speech tags, and `heads` is the list of head indices
        (one head index per word in the tree).
    """
    input_data = nlp4.conllu(source)
    
    for w_info in input_data:
        words =['<ROOT>']
        tags = ['<ROOT>']
        idx = [0]
        for e in w_info:
            words.append(e[1])
            tags.append(e[3])
            idx.append(int(e[6]))
        yield (words,tags,idx)
    
    

Test your code by executing the following code cell. When you are done with Problem&nbsp;1, this should produce exactly the same output as the call to `nlp4.trees()` above:

In [7]:
with open("/home/TDDE09/labs/l4/data/en-ud-dev.conllu") as fp:
    print(next(trees(fp)))

(['<ROOT>', 'From', 'the', 'AP', 'comes', 'this', 'story', ':'], ['<ROOT>', 'ADP', 'DET', 'PROPN', 'VERB', 'DET', 'NOUN', 'PUNCT'], [0, 3, 3, 4, 0, 6, 4, 4])


## The multi-class perceptron classifier

The first core component of the baseline system is a multi-class perceptron classifier, very much like the one that you have implemented in lab&nbsp;1 (on text classification), and then again in lab&nbsp;3 (on part-of-speech tagging).

### Implementation

In [8]:
class Perceptron(nlp4.Perceptron):
    """A multi-class perceptron classifier.
    
    A multi-class perceptron consists of a number of cells, one cell
    for each class. When a cell is presented with an input, it gets
    activated, and the cell with the highest activation predicts the
    class for the input. An input is represented by a feature vector,
    and the activation is computed by taking the dot product (weighted
    sum) of this feature vector and the cell-specific weight vector.
    
    This implementation of a multi-class perceptron assumes that both
    classes and features can be used as dictionary keys. Feature
    vectors are represented as lists of features.
    
    Attributes:
        classes: A set containing the classes of this classifier.
        weights: A nested dictionary mapping feature–class
            combinations to class-specific feature weights (floats).
    """

    def __init__(self):
        """Initialises a new classifier."""
        super().__init__()

    def predict(self, x, candidates=None):
        """Predicts the class for a feature vector.
        
        This computes the activations for the classes of this
        perceptron for the feature vector `x` and returns the class
        with the highest activation.
        
        Args:
            x: A feature vector (a list of features).
            candidates: A list of candidate classes, or `None` if all
                classes defined in this perceptron should be
                considered as candidates.
        
        Returns:
            The candidate class with the highest activation. If
            several classes have the same activation, returns the
            class that is largest with respect to the standard
            ordering on classes.
        """
        return super().predict(x, candidates)

    def update(self, x, y):
        """Updates the weight vectors with a single training instance.
        
        Args:
            x: A feature vector.
            y: The gold-standard class for the specified feature
                vector.
        
        Returns:
            The predicted class for the specified feature vector.
        """
        return super().update(x, y)

    def finalize(self):
        """Averages the weight vectors."""
        super().finalize()

The only novelty of this implementation compared to the ones that you have produced earlier is in the `predict()` method. This method now takes an optional argument `candidates`, which may be either `None` (the default value) or a list of classes. When it is the latter, the perceptron restricts its prediction to the specified classes, that is, it predicts the highest-scoring class from the list, rather than the highest-scoring class overall. This functionality will be useful in the parser.

## The part-of-speech tagger

The second core component of the baseline system is a part-of-speech tagger, very similar to the one that you have implemented in lab&nbsp;3.

### Implementation

In [9]:
class Tagger(nlp4.Tagger):
    """A part-of-speech tagger based on a multi-class perceptron
    classifier.
    
    This tagger implements a simple, left-to-right tagging algorithm
    where the prediction of the tag for the next word in the sentence
    can be based on the surrounding words and the previously
    predicted tags. The exact features that this prediction is based
    on can be controlled with the `features()` method, which should
    return a feature vector that can be used as an input to the
    multi-class perceptron.
    
    Attributes:
        classifier: A multi-class perceptron classifier.
    """
    
    def __init__(self):
        """Initialises a new tagger."""
        super().__init__()
    
    def tag(self, words):
        """Tags a sentence with part-of-speech tags.
        
        Args:
            words: The input sentence, a list of words.
        
        Returns:
            The list of predicted tags for the input sentence.
        """
        return super().tag(words)
    
    def update(self, words, gold_tags):
        """Updates the tagger with a single training example.
        
        Args:
            words: The list of words in the input sentence.
            gold_tags: The list of gold-standard tags for the input
                sentence.
        
        Returns:
            The list of predicted tags for the input sentence.
        """
        return super().update(words, gold_tags)
    
    def features(self, words, i, pred_tags):
        """Extracts features for the specified tagger configuration.
        
        Args:
            words: The input sentence, a list of words.
            i: The index of the word that is currently being tagged.
            pred_tags: The list of previously predicted tags.
        
        Returns:
            A feature vector for the specified configuration.
        """
        return super().features(words, i, pred_tags)
    
    def finalize(self):
        """Averages the weight vectors."""
        super().finalize()

SyntaxError: invalid syntax (<ipython-input-9-ff71de105848>, line 55)

The reference implementation extracts the following features: current word, previous word, next word, and previous tag.

## The parser

Your main task in this lab is to implement the third and final core component of the baseline system, the parser.

<div class="panel panel-primary">
<div class="panel-heading">Problem 2</div>
<div class="panel-body">
Implement a transition-based dependency parser, train it on the specified training data, and evaluate its performance on the development data. Your parser should get the same results as the reference implementation.
</div>
</div>

### Implementation

Your starting point for this problem is the following skeleton class:

In [ ]:
class Parser(nlp4.Parser):
    """A transition-based dependency parser.
    
    This parser implements the arc-standard algorithm for dependency
    parsing. When being presented with an input sentence, it first
    tags the sentence for parts of speech, and then uses a multi-class
    perceptron classifier to predict a sequence of *moves*
    (transitions) that construct a dependency tree for the input
    sentence. Moves are encoded as integers as follows:
    
    SHIFT = 0, LEFT-ARC = 1, RIGHT-ARC = 2
    
    At any given point in the predicted sequence, the state of the
    parser can be specified by: the index of the first word in the
    input sentence that the parser has not yet started to process; a
    stack holding the indices of those words that are currently being
    processed; and a partial dependency tree, represented as a list of
    indices such that `tree[i]` gives the index of the head (parent
    node) of the word at position `i`, or 0 in case the corresponding
    word has not yet been assigned a head.
    
    Attributes:
        tagger: A part-of-speech tagger.
        classifier: A multi-class perceptron classifier used to
            predict the next move of the parser.
    """
    
    def __init__(self):
        """Initialises a new parser."""
        self.tagger = Tagger()
        self.perceptron = Perceptron()
        
    
    def parse(self, words):
        """Parses a sentence.
        
        Args:
            words: The input sentence, a list of words.
        
        Returns:
            A pair consisting of the predicted tags and the predicted
            dependency tree for the input sentence.
        """
        word_tags = self.tagger.tag(words)
        stack = []
        tree = [0 for _ in range(len(words))]
        idx = 0
        
        while True:
            possible_moves = self.valid_moves(idx, stack, tree)
            if not possible_moves:
                break
            feature_list = self.features(words, word_tags, idx, stack, tree)
            move = self.perceptron.predict(feature_list, possible_moves)
            idx,stack,tree = self.move(idx,stack,tree,move)
        
        return (word_tags,tree)
        
    
    
    def valid_moves(self, i, stack, pred_tree):
        """Returns the valid moves for the specified parser
        configuration.
        
        Args:
            i: The index of the first unprocessed word.
            stack: The stack of words (represented by their indices)
                that are currently being processed.
            pred_tree: The partial dependency tree.
        
        Returns:
            The list of valid moves for the specified parser
                configuration.
        """
        moves = []
        if i < 2:
            return [0]
        elif i < (len(pred_tree)):
            moves.append(0)
            
        if len(stack) > 1:
            moves.append(2)
        if len(stack) > 2:
            moves.append(1)
        return moves
    
    
    def move(self, i, stack, pred_tree, move):
        """Executes a single move.
        
        Args:
            i: The index of the first unprocessed word.
            stack: The stack of words (represented by their indices)
                that are currently being processed.
            pred_tree: The partial dependency tree.
            move: The move that the parser should make.
        
        Returns:
            The new parser configuration, represented as a triple
            containing the index of the new first unprocessed word,
            stack, and partial dependency tree.
        """
        if move == 0:
            stack.append(i)
            i += 1
        elif move == 1:
            pred_tree[stack.pop(-2)] = stack[-1]
        else:
            pred_tree[stack.pop(-1)] = stack[-2]
        return i,stack,pred_tree
    
    def update(self, words, gold_tags, gold_tree):
        """Updates the move classifier with a single training
        instance.
        
        Args:
            words: The input sentence, a list of words.
            gold_tags: The list of gold-standard tags for the input
                sentence.
            gold_tree: The gold-standard tree for the sentence.
        
        Returns:
            A pair consisting of the predicted tags and the predicted
            dependency tree for the input sentence.
        """
        pred_tags = self.tagger.update(words,gold_tags)
        stack = []
        tree = [0 for _ in range(len(gold_tree))]
        idx = 0
        
        while True:
            possible_moves = self.valid_moves(idx, stack, tree)
            if not possible_moves:
                break
            gold_move = self.gold_move(idx, stack,tree, gold_tree)
            
            # If we use pred_tags instead of gold_tags here when we create
            # our feature_list we get the same result as the reference implementation. 
            # We found that gold_tags gave improved accuracy so we decided to keep it like this
            feature_list = self.features(words, gold_tags, idx, stack, tree)
            
            self.perceptron.update(feature_list,gold_move)
            idx,stack,tree = self.move(idx,stack,tree,gold_move)
        
        return gold_tags,gold_tree

    
    def gold_move(self, i, stack, pred_tree, gold_tree):
        """Returns the gold-standard move for the specified parser
        configuration.
        
        The gold-standard move is the first possible move from the
        following list: LEFT-ARC, RIGHT-ARC, SHIFT. LEFT-ARC is
        possible if the topmost word on the stack is the gold-standard
        head of the second-topmost word, and all words that have the
        second-topmost word on the stack as their gold-standard head
        have already been assigned their head in the predicted tree.
        Symmetric conditions apply to RIGHT-ARC. SHIFT is possible if
        at least one word in the input sentence still requires
        processing.
        
        Args:
            i: The index of the first unprocessed word.
            stack: The stack of words (represented by their indices)
                that are currently being processed.
            pred_tree: The partial dependency tree.
            gold_tree: The gold-standard dependency tree.
        
        Returns:
            The gold-standard move for the specified parser
            configuration, or `None` if no move is possible.
        """
        if len(stack) > 1 and stack[-1] == gold_tree[stack[-2]]:
            is_head = sum([1 for h in gold_tree if h == stack[-2]])
            is_pred_head = sum([1 for h in pred_tree if h == stack[-2]])
            if is_head == is_pred_head:
                return 1
            
        if len(stack) > 1 and stack[-2] == gold_tree[stack[-1]]:
            is_head = sum([1 for h in gold_tree if h == stack[-1]])
            is_pred_head = sum([1 for h in pred_tree if h == stack[-1]])
            if is_head == is_pred_head:
                return 2
        
        if i < len(gold_tree):
            return 0
        
        return None
        
    
    def features(self, words, tags, i, stack, parse):
        """Extracts features for the specified parser configuration.

        Args:
            words: The input sentence, a list of words.
            gold_tags: The list of gold-standard tags for the input
                sentence.
            i: The index of the first unprocessed word.
            stack: The stack of words (represented by their indices)
                that are currently being processed.
            parse: The partial dependency tree.

        Returns:
            A feature vector for the specified configuration.
        """
        feat_list = []
        if i > (len(words) -1):
            feat_list = ['currentword:None','currenttag:None']
        else:
            feat_list.append('currentword:%s' % words[i])
            feat_list.append('currenttag:%s' % tags[i])
        
        if stack:
            feat_list.append('lastword:%s' % words[stack[-1]])
            feat_list.append('lasttag:%s' % tags[stack[-1]])
        
            if len(stack) > 1:
                feat_list.append('lastlastword:%s' % words[stack[-2]])
                feat_list.append('lastlasttag:%s' % tags[stack[-2]])
            else:
                feat_list.append('lastlastword:None')
                feat_list.append('lastlasttag:None')
        else:
            feat_list.append('lastword:None')
            feat_list.append('lasttag:None')
            feat_list.append('lastlastword:None')
            feat_list.append('lastlasttag:None')
        return feat_list
        
    
    def finalize(self):
        """Averages the weight vectors."""
        self.tagger.finalize()
        self.perceptron.finalize()


### Comments

#### Main parsing method

The first step in the parsing is to tag the sentence for parts of speech. After tagging, the `parse()` method initialises a new parser configuration and asks the move classifier to predict the move that the parser should make in that configuration. It is important that this prediction is restricted to predict a valid move; this is where the optional argument of the `predict()` method in the multi-class perceptron becomes relevant (see above). Making the predicted move results in a new configuration, for which the process is iterated; this continues as long as there are valid moves left to make.

#### Extract features

The reference implementation extracts the following features:

* word form of the next word to be processed
* tag of the next word to be processed
* word form of the topmost word on the stack
* tag of the topmost word on the stack
* word form of the second-topmost word on the stack
* tag of the second-topmost word on the stack

In your own implementation, make sure to handle the special cases where there are no more words left that need to be processed, or the stack is empty.

## Evaluate the parser

Use the following code to train and evaluate your parser. You can control the number of examples used for training by setting the value `n_examples`.

In [ ]:
n_examples = 200    # Set to None to train on all examples

parser = Parser()
with open("/home/TDDE09/labs/l4/data/en-ud-train-projective.conllu") as fp:
    for i, (words, gold_tags, gold_tree) in enumerate(trees(fp)):
        parser.update(words, gold_tags, gold_tree)
        print("\rUpdated with sentence #{}".format(i), end="")
        if n_examples and i >= n_examples:
            break
    print("")
parser.finalize()

acc_k = acc_n = 0
uas_k = uas_n = 0
with open("/home/TDDE09/labs/l4/data/en-ud-dev.conllu") as fp:
    for i, (words, gold_tags, gold_tree) in enumerate(trees(fp)):
        pred_tags, pred_tree = parser.parse(words)
        acc_k += sum(int(g == p) for g, p in zip(gold_tags, pred_tags)) - 1
        acc_n += len(words) - 1
        uas_k += sum(int(g == p) for g, p in zip(gold_tree, pred_tree)) - 1
        uas_n += len(words) - 1
        print("\rParsing sentence #{}".format(i), end="")
    print("")
print("Tagging accuracy: {:.2%}".format(acc_k / acc_n))
print("Unlabelled attachment score: {:.2%}".format(uas_k / uas_n))

With your own implementation you should obtain scores very similar to the ones of the reference implementation.